In [3]:
# initial setup
try:
    # settings colab:
    import google.colab
    
    # si usan colab, deben cambiar el token de esta url
    #! mkdir -p ../data
    # los que usan colab deben modificar el token de esta url:
    #! wget -O ../data/properati_2017.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_07_Pandas_2/Data/resultado-de-encuestas-2017-2018.csv?token=AA4GFHO3EKGWGILBCV4BISC6WR6B4
    
except ModuleNotFoundError:    
    # settings local:
    %run "common/base_setup.py"

Running command `conda list`... ok
pandas=1.0.3 already installed
matplotlib=2.2.2 already installed
bokeh=2.0.0 already installed
seaborn=0.10.0 already installed
ipywidgets=7.5.1 already installed
pytest=5.3.4 already installed
chardet=3.0.4 already installed
plotly=4.6.0 already installed
Running command `conda install --yes chart-studio=1.1.0`... error
    
PackagesNotFoundError: The following packages are not available from current channels:

  - chart-studio=1.1.0

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.





## Carga Inicial del Dataset

In [45]:
import pandas as pd
import numpy as np

In [46]:
data   = pd.read_csv("./data/properati-fase3.csv", sep = ",", low_memory=False)
subtes = pd.read_csv("./data/estaciones-de-subte.csv", sep = ",", low_memory=False)

data["c_lat"] = data["c_lat"].astype(np.float)
data["c_lng"] = data["c_lng"].astype(np.float)

## Aplicacion de distancias al subte

**NOTA**: esta distancia sólo la vamos a calcular para *Capital Federal*.

In [30]:
def haversine(Olat, Olon, Dlat, Dlon):
    radius = 6371.  # km
    d_lat = np.radians(Dlat - Olat)
    d_lon = np.radians(Dlon - Olon)
    a = (np.sin(d_lat / 2.) * np.sin(d_lat / 2.) +
         np.cos(np.radians(Olat)) * np.cos(np.radians(Dlat)) *
         np.sin(d_lon / 2.) * np.sin(d_lon / 2.))
    c = 2. * np.arctan2(np.sqrt(a), np.sqrt(1. - a))
    d = radius * c
    return d

def search_closest_distance(datasubte, lat, long):
    datasubte['dist'] = datasubte.apply(lambda x: haversine(x['lat'], x['long'], lat, long), axis=1)
    return datasubte['dist'].min()

Aplicamos efectivamente el cálculo de la distancia al subte.

**NOTA**: Este proceso es muy pesado.

In [31]:
def apply_distance(x):
    if x['c_lat'] is np.nan:
        return 0
    if x['c_lng'] is np.nan:
        return 0
    if x['c_estado'] != 'Capital Federal':
        return 0
    return search_closest_distance(subtes, x['c_lat'], x['c_lng'])
data['dist_subte'] = data.apply(apply_distance, axis=1)

Finalmente, visualizamos algunos de los resultados:

In [42]:
subtes_mask = data['c_lat'].notna() & data['c_lng'].notna() & (data['c_estado'] == 'Capital Federal')
data.loc[subtes_mask, ['id', 'c_lat', 'c_lng', 'dist_subte']].head()

,id,c_lat,c_lng,dist_subte
0,0,-34.661824,-58.508839,4.781915
1,2,-34.652262,-58.522982,5.436760
4,7,-34.559873,-58.443362,1.068329
5,8,-34.559873,-58.443362,1.068329
7,13,-34.588560,-58.430530,1.167057


Y corroboramos que a aquellos registros que no son de *Capital Federal* les hayamos asignado **0**.

In [43]:
nosubtes_mask = data['c_lat'].notna() & data['c_lng'].notna() & (data['c_estado'] != 'Capital Federal')
data.loc[nosubtes_mask, ['id', 'c_lat', 'c_lng', 'dist_subte']].head()

,id,c_lat,c_lng,dist_subte
2,4,-38.002626,-57.549447,0.0
3,6,-34.532957,-58.521782,0.0
6,12,-34.486019,-58.503814,0.0
9,15,-34.587911,-58.698783,0.0
11,17,-34.408717,-58.634389,0.0


In [44]:
data.to_csv('./data/properati-fase4.csv', sep=',', index=False)